In [23]:
import os

from os.path import  join, isdir
from plan import Plan
from action import Action
from utils import load_from_folder
from multiprocess import Pool
import random
from logging import exception


In [40]:
save_dir = './new_plans/'
data_base_dir = '../datasets/'
domain = 'logistics'
results_dir = f"{save_dir}/{domain}/"   
source_dir = f"{join(data_base_dir, domain)}/optimal_plans/dictionaries_and_plans/" 
print('Domain dir:', source_dir)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

plans_to_process = 10
versions_per_plan = 3
number_of_goal = 3

Domain dir: ../datasets/logistics/optimal_plans/dictionaries_and_plans/


In [7]:
splits = {}
plans = load_from_folder(source_dir,["plans"])[0]
print(f"Plans: {len(plans)}")


plans loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans/
Plans: 47769


In [38]:
plans[0].initial_state

['at tru2 pos44',
 'at tru4 pos22',
 'at tru1 pos66',
 'at tru5 pos23',
 'at tru3 pos21',
 'at obj66 pos33',
 'at obj13 pos13',
 'at obj11 pos11',
 'at obj12 pos11',
 'at obj00 pos44',
 'at obj23 pos33',
 'at obj44 pos66',
 'at obj55 pos21',
 'at obj88 pos55',
 'at obj33 pos44',
 'at apn6 apt7',
 'at apn3 apt7',
 'at apn2 apt7',
 'at apn1 apt7',
 'at apn7 apt6',
 'at apn5 apt6']

In [ ]:
#obj_numbers_list = []
for plan in plans:
    obj_set = set()
    package_for_goal_set = set()
    pos_for_goal_set = set()
    
    #find all objects in the initial state and actions
    for text in plan.initial_state:
        for obj in text.split(" ")[1:]:
            obj_set.add(obj)
    for action in plan.actions:
        for fluent in action.positiveEffects:
            for obj in fluent.split(" ")[1:]:
                obj_set.add(obj)
        for fluent in action.negativeEffects:
            for obj in fluent.split(" ")[1:]:
                obj_set.add(obj)
    #obj_numbers_list.append(len(obj_set))
    
    #find all objects and split them in their types
    pos_set = set()
    apn_set = set()
    cit_set = set()
    apt_set = set()
    tru_set = set()
    pack_set = set()
    for obj in obj_set:
        if obj.startswith("pos"):
            pos_set.add(obj)
            pos_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("obj"):
            pack_set.add(obj)
            package_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("apn"):
            apn_set.add(obj)
        elif obj.startswith("cit"):
            cit_set.add(obj)
        elif obj.startswith("tru"):
            tru_set.add(obj)
        elif obj.startswith("apt"):
            apt_set.add(obj)
            
    
    #raise an exception if number of goals > number of packages or positions
    if number_of_goal > len(package_for_goal_set):
        raise exception(f"Number of goals {number_of_goal} is greater than the number of objects {len(package_for_goal_set)}")

    if number_of_goal > len(pos_for_goal_set):
        raise exception(f"Number of goals {number_of_goal} is greater than the number of positions {len(pos_for_goal_set)}")

    #now we start creating the new plans from the current one
    for i in range(0, versions_per_plan):
        #create a set of goals of the type [at obj pos] 
        #by picking a random object and a random position from the sets
        #and removing them from the sets so they won't be used again
        goal_set= set()    
        for i in range(0, number_of_goal):
            random_package = random.choice(list(package_for_goal_set))
            package_for_goal_set.remove(random_package) 
            random_pos = random.choice(list(pos_for_goal_set))
            pos_for_goal_set.remove(random_pos)
            goal_set.add(f"at {random_package} {random_pos}") 
        print(f"Goal set: {goal_set}")
    
        #now we build the new pddl file with these goals
        new_problem = f"(define (problem new_problem)\n(:domain {domain})\n(:objects\n"
        new_problem += f"  "
        if len(apn_set) > 0:
            for apn in apn_set:
                new_problem += f"{apn} "
            new_problem += f"- airplane\n"
            new_problem += f"  "
        if len(cit_set) > 0:
            for cit in cit_set:
                new_problem += f"{cit} "
            new_problem += f"- city\n"
            new_problem += f"  "
        if len(apt_set) > 0:
            for apt in apt_set:
                new_problem += f"{apt} "
            new_problem += f"- airport\n"
            new_problem += f"  "
        if len(tru_set) > 0:
            for tru in tru_set:
                new_problem += f"{tru} "
            new_problem += f"- truck\n"
            new_problem += f"  "
        if len (pack_set) > 0:
            for pack in pack_set:
                new_problem += f"{pack} "
            new_problem += f"- package\n"
            new_problem += f"  "
        if len(pos_set) > 0:
            for pos in pos_set:
                new_problem += f"{pos} "
            new_problem += f"- location\n"
            new_problem += f"  "
        new_problem += f")\n"
        
        new_problem += f"(:init\n"
        for fluent in plan.initial_state:
            new_problem += f"   {fluent}\n"
        new_problem += f")\n"
        
        new_problem += f"(:goal (and\n"
        for goal in goal_set:
            new_problem += f"   {goal}\n"
        new_problem += f"))\n)"
        print(new_problem + "\n\n")
        #todo save the new problem in a file
    break



Goal set: {'at obj88 pos33', 'at obj12 pos21', 'at obj44 pos23'}
(define (problem new_problem)
(:domain logistics)
(:objects
  apn7 apn6 apn1 apn5 apn3 apn2 - airplane
  apt7 apt8 apt6 apt5 - airport
  tru4 tru3 tru1 tru2 tru5 - truck
  obj33 obj11 obj12 obj44 obj13 obj55 obj00 obj88 obj66 obj23 - package
  pos33 pos55 pos22 pos44 pos21 pos13 pos66 pos23 pos11 - location
  )
(:init
   at tru2 pos44
   at tru4 pos22
   at tru1 pos66
   at tru5 pos23
   at tru3 pos21
   at obj66 pos33
   at obj13 pos13
   at obj11 pos11
   at obj12 pos11
   at obj00 pos44
   at obj23 pos33
   at obj44 pos66
   at obj55 pos21
   at obj88 pos55
   at obj33 pos44
   at apn6 apt7
   at apn3 apt7
   at apn2 apt7
   at apn1 apt7
   at apn7 apt6
   at apn5 apt6
)
(:goal (and
   at obj88 pos33
   at obj12 pos21
   at obj44 pos23
))
)
(define (problem new_problem)
(:domain logistics)
(:objects
  apn7 apn6 apn1 apn5 apn3 apn2 - airplane
  apt7 apt8 apt6 apt5 - airport
  tru4 tru3 tru1 tru2 tru5 - truck
  obj33 obj